# Queens Problem

### importem llibreries que utilitzarem i definim variables globals


In [2]:
import numpy as np
import random

INITIAL_POPULATION = 10
BOARD_SIZE = 8
MUTATION_CHANCE = 0.1

### Definir Agent, població i display

In [3]:
def init_agent(board_size):
    return random.sample(range(board_size), board_size)

def display_agent(agent):
    for queenPosition in agent:
        result = []

        for el in range(len(agent)):
            result.append(1 if el == queenPosition else 0)

        print(result)

def init_population(initial_population_size, board_size):
    return [init_agent(board_size) for _ in range(initial_population_size)]


_population = init_population(INITIAL_POPULATION, BOARD_SIZE)
_agent = _population[0]
assert BOARD_SIZE == len(_agent)
assert INITIAL_POPULATION == len(_population)
display_agent(_agent)
# assert BOARD_SIZE*BOARD_SIZE == len(display_agent(_agent))

[0, 0, 0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]


### definir crossover

In [4]:
def crossover(parent1, parent2):
    board_size = len(parent1)

    crossover_point = random.randint(1, board_size - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]

    # Remove duplicates
    unique_values = set(child)
    all_values = set(range(board_size))
    missing_values = list(all_values - unique_values)

    for i in range(board_size):
        if child.count(child[i]) > 1:
            child[i] = missing_values.pop(0)

    return child

child = crossover(_population[0], _population[1])
print(_population[0])
print(_population[1])
print(child)

[3, 1, 4, 7, 0, 6, 2, 5]
[2, 3, 6, 4, 0, 5, 1, 7]
[3, 1, 4, 5, 0, 6, 2, 7]


### definir mutació


In [5]:
def mutation(agent):
    pos1, pos2 = random.sample(range(len(agent)), 2)
    agent[pos1], agent[pos2] = agent[pos2], agent[pos1]
    return agent

def mutation_with_chance(agent, mutation_chance):
    if random.random() < mutation_chance:
        return mutation(agent)

    return agent

_agentCopy = _agent.copy()
print(mutation_with_chance(_agentCopy, 0.1))
print(_agent)

[3, 1, 4, 7, 0, 6, 2, 5]
[3, 1, 4, 7, 0, 6, 2, 5]


### definir crossover de tota la població amb mutació

In [6]:
def replicate(population, mutation_chance):
    new_population = []
    population_size = len(population)

    # Generate childs
    for _ in range(population_size // 2):
        parent1, parent2 = random.sample(population, 2)

        child1 = crossover(parent1, parent2)
        child2 = crossover(parent2, parent1)

        child1 = mutation_with_chance(child1, mutation_chance)
        child2 = mutation_with_chance(child2, mutation_chance)

        new_population.append(child1)
        new_population.append(child2)

    # Generate one more child if is necesary
    if population_size % 2 == 1:
        parent1, parent2 = random.sample(population, 2)
        child = crossover(parent1, parent2)
        child = mutation_with_chance(child, mutation_chance)
        new_population.append(child)

    # Return same size that population input
    return random.sample(new_population, population_size)

replicate(_population, MUTATION_CHANCE)


[[4, 2, 0, 5, 3, 1, 7, 6],
 [4, 2, 6, 1, 7, 3, 0, 5],
 [4, 1, 3, 7, 0, 6, 2, 5],
 [3, 0, 2, 4, 6, 1, 7, 5],
 [2, 1, 4, 6, 7, 3, 0, 5],
 [4, 2, 6, 1, 7, 0, 3, 5],
 [1, 6, 0, 4, 3, 2, 5, 7],
 [2, 3, 6, 4, 0, 5, 1, 7],
 [2, 4, 6, 1, 7, 3, 0, 5],
 [2, 3, 1, 4, 0, 5, 7, 6]]

### definir fitness function

mirar diagonals i treure repetides

In [7]:
def fitness_agent(agent):
    conflicts = 0
    for i in range(len(agent)):
        for j in range(i + 1, len(agent)):
            if abs(agent[i] - agent[j]) == j - i:
                conflicts += 1
    return conflicts

def fitness_population(population):
    return [fitness_agent(agent) for agent in population]

display_agent(_agent)
print(fitness_agent(_agent))
assert fitness_agent(_agent) <= BOARD_SIZE

[0, 0, 0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
1


### Population Selection (by score)


In [8]:
def sort_population(population, fitness_list):
    paired = list(zip(population, fitness_list))
    sorted_paired = sorted(paired, key=lambda x: x[1], reverse=False)
    return [x[0] for x in sorted_paired]

def select_population(population, fitness_list, population_size):
    sorted_population = sort_population(population, fitness_list)
    return sorted_population[:population_size]

print(select_population(_population, fitness_population(_population), BOARD_SIZE))
print(_population)

[[3, 1, 4, 7, 0, 6, 2, 5], [3, 0, 2, 4, 6, 1, 7, 5], [1, 7, 4, 3, 6, 0, 5, 2], [4, 2, 0, 5, 3, 1, 7, 6], [2, 3, 6, 4, 0, 5, 1, 7], [7, 2, 6, 4, 1, 0, 3, 5], [1, 0, 4, 7, 2, 5, 3, 6], [4, 7, 2, 5, 0, 6, 1, 3]]
[[3, 1, 4, 7, 0, 6, 2, 5], [2, 3, 6, 4, 0, 5, 1, 7], [7, 2, 6, 4, 1, 0, 3, 5], [4, 7, 2, 5, 0, 6, 1, 3], [4, 2, 6, 1, 7, 3, 0, 5], [1, 0, 4, 7, 2, 5, 3, 6], [1, 7, 4, 3, 6, 0, 5, 2], [3, 0, 2, 4, 6, 1, 7, 5], [0, 1, 6, 4, 3, 2, 5, 7], [4, 2, 0, 5, 3, 1, 7, 6]]


### definir main

(seguir els pasos de classe)

In [9]:
def main(population_size, board_size, mutation_chance):
    population = init_population(population_size, board_size)
    generation = 0

    while True:
        fitness_list = fitness_population(population)
        best_fitness = min(fitness_list)
        best_agent = population[fitness_list.index(best_fitness)]

        if generation % 100 == 0:
            print(f"Population {generation}: {best_fitness}")

        if best_fitness == 0:
            print(f"Solución óptima encontrada en la generación {generation}: {best_agent}")
            break

        new_population = replicate(population, mutation_chance)
        new_fitness_list = fitness_population(new_population)

        combined_population = population + new_population
        combined_fitness_list = fitness_list + new_fitness_list
        selected_population = select_population(combined_population, combined_fitness_list, population_size)

        population = selected_population
        generation += 1

    return best_agent

### Provar el codi

In [10]:
POPULATION_SIZE = 100
BOARD_SIZE = 100
MUTATION_CHANCE = 0.2

agent = main(POPULATION_SIZE, BOARD_SIZE, MUTATION_CHANCE)
display_agent(agent)

Population 0: 48
